## Auto mode

This is for testing propouses

In [ ]:
from config import Color
from helpers import clean  #, local2global
from mavlink.customtypes.location import ENUPose, GRAPose
from plan import Plan
from simulator import QGC, ConfigGazebo, ConfigQGC, Gazebo, NoneVisualizer, Simulator

clean()

## Simulation Configuration

In [ ]:
mission_name = "square"

gra_origin = GRAPose(-35.3633280, 149.1652241, 0, 90)   # east, north, up, heading
enu_origin = ENUPose(0, 0, gra_origin.alt, gra_origin.heading) 

base_homes= ENUPose.list([  # east, north, up, heading
    (0., 0., 0., 0.),
    (15., 0., 0., 0),
    (-15., -15., 0., 0),
    (-15., 0., 0., 45),
    (5., -20., 0., 0.),
])
n_uavs = len(base_homes)
base_paths = [Plan.create_square_path(side_len=10, alt=5,heading=0) for _ in base_homes]

colors=[
    Color.BLUE,
    Color.GREEN,
    Color.BLACK,
    Color.ORANGE,
    Color.RED,
]

## Gazebo Congiguration

In [ ]:
gaz_config = ConfigGazebo(origin= enu_origin,
                          world_path="simulator/gazebo/worlds/runway.world")

for path,home,c in zip(base_paths,base_homes,colors):
    gaz_config.add(base_path=path,base_home=home,color=c)
gaz_config.show()

## QGroundControl Configuration

In [ ]:
qgc_config = ConfigQGC(origin= gra_origin)

for path,home,c in zip(base_paths,base_homes,colors):
    qgc_config.add(base_path=path,base_home=home,color=c)
qgc_config.show()

## Save the missions

In [ ]:
qgc_config.save_missions('square')

## Visualization Parameters

In [ ]:
novis=NoneVisualizer()

gaz= Gazebo(gaz_config,gra_origin)

qgc=QGC(qgc_config)

## Launch Simulatotion

In [ ]:
# TODO: Is it possible/feasible use more than one visualizer at the same time? 
# It seems there is a port conflict, only the visualizer that launches first works.

simulator = Simulator(
	visualizers=[qgc],
	mission_names=[f"square_{i + 1}" for i in range(n_uavs)],
	terminals=False,
	verbose=1,
)
orac = simulator.launch(gcs_sysids={'blue 🟦': [1]})

## Oracle checking

In [ ]:
while len(orac.conns):
    for sysid in list(orac.conns.keys()):
        if orac.is_plan_done(sysid):
            orac.remove(sysid)